In [13]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

In [2]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-12-07 21:11:40--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-12-07 21:11:40 (17.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
# !pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 15.1 MB/s eta 0:00:00a 0:00:01


In [8]:
import tiktoken
enc = tiktoken.get_encoding("o200k_base")

In [21]:
# config
block_size = 256

In [18]:
data = torch.tensor(enc.encode(text))
train_data = data[:int(0.9*len(data))]
test_data = data[int(0.9*len(data)):]

In [22]:
class GPTDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data) - block_size - 1

    def __getitem__(self, idx):
        return {"x": self.data[idx:idx+block_size], "y": self.data[idx+1:idx+block_size+1]}

In [23]:
train_ds = GPTDataset(train_data)
test_ds = GPTDataset(test_data)

In [25]:
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=64, shuffle=True)

In [26]:
for batch in train_dl:
    print(batch)
    break

{'x': tensor([[ 65865,    316,    261,  ...,    285,   5119,    461],
        [    88,     25,    889,  ...,   1485,   6354,    289],
        [  7994,    558,  16936,  ...,    198,   2886,  10143],
        ...,
        [120206,     11,    326,  ...,    395,    495,   4175],
        [  1584,  81238,  84479,  ...,   9144,    540,    461],
        [ 14347,   1280,    963,  ...,   2212,    873,     11]]), 'y': tensor([[   316,    261, 111761,  ...,   5119,    461, 104853],
        [    25,    889,    922,  ...,   6354,    289,    412],
        [   558,  16936,     11,  ...,   2886,  10143,   4465],
        ...,
        [    11,    326,   2242,  ...,    495,   4175,  37788],
        [ 81238,  84479,    734,  ...,    540,    461,     82],
        [  1280,    963,   3901,  ...,    873,     11,  17291]])}
